In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import fingertips_py as ftp
import pandas as pd
import numpy as np
import random
import csv
import requests
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
#list of health indicators to fetch values for from Public Health England's Fingertips data source
indicator_list = ['90729','90742','90759','90790','91046','91521','91361','92326','92670','92841','93119','93124','93316','93355',
'93356','93379','93380','93357','93456','93466','93515','93604','92621','11401','20101','40401','40701','90443',
'90669','91269','91409','91743','92199','92432','92924','93015','93077','93093','93347','93377','93439','93440',
'93563','40501','40502']

In [ ]:
#These ones seem to be having access problems
'91743','92199','40502'

In [3]:
#load district codes
area_codes = pd.read_csv("area_codes.csv")
district_area_codes = area_codes["Area Code"].tolist()
number_districts = len(district_area_codes)

#use the area code dataframe as a base on which to join the health indicator data as it's fetched
data = area_codes.copy()

#preserve full feature descriptions
with open('feature_descriptions.csv', 'a', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    header = ['indicator','description']
    writer.writerow(header)

    #fetch each of the health indicator value sets (for year 2020) and join to dataframe
    for indicator in indicator_list:
        temp_data = ftp.retrieve_data.get_all_data_for_indicators(indicators = indicator, area_type_id = 101,
                                                             filter_by_area_codes = district_area_codes)
        if(len(temp_data)==0):
            print('no data found for: ',indicator)
            continue
        else:
            writer.writerow([indicator,temp_data['Indicator Name']])
            time_period = temp_data.loc[len(temp_data)-1,'Time period']
            temp_data = temp_data[temp_data['Time period']==time_period]
            #don't break out by gender
            temp_data = temp_data.loc[(temp_data["Sex"]== "Persons") | (temp_data["Sex"]== "Not applicable")]
            temp_data = temp_data[['Area Code','Value']]
            temp_data.rename(columns={'Value':indicator}, inplace=True)
            data = pd.merge(data, temp_data, on='Area Code')


C:\ProgramData\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3364: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


no data found for:  91743
no data found for:  92199


C:\ProgramData\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3364: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


no data found for:  40502


In [60]:
descriptions = pd.read_csv('feature_descriptions.csv')

In [61]:
descriptions

,indicator,description
0,90729,"0 Genital warts diagnostic rate / 100,00..."
1,90742,"0 Syphilis diagnostic rate / 100,000\n1 ..."
2,90759,"0 Gonorrhoea diagnostic rate / 100,000\n..."
3,90790,"0 HIV diagnosed prevalence rate per 1,00..."
4,91046,"0 Genital herpes diagnosis rate / 100,00..."
5,91521,"0 Chlamydia diagnostic rate / 100,000\n1..."
6,91361,0 TB incidence (three year average)\n1 ...
7,92326,0 Mortality rate from a range of specif...
8,92670,0 Adjusted antibiotic prescribing in pri...
9,92841,0 Typhoid & paratyphoid incidence rate/1...


In [62]:
#view head of output
pd.set_option('max_columns',None)
data.head(200)

,Area Code,Area Name,90729,90742,90759,90790,91046,91521,91361,92326,92670,92841,93119,93124,93316,93355,93356,93379,93380,93357,93456,93466,93515,93604,92621,11401,20101,40401,40701,90443,90669,91269,91409,92432,92924,93015,93077,93093,93347,93377,93439,93440,93563,40501
0,E06000001,Hartlepool,35.167740,11.722580,54.350143,0.875074,43.693253,346.348938,3.572398,9.310346,0.780798,0.00000,0.000000,0.000000,2.144956,16.160833,127.131883,16.160833,8.619111,2.15,0.000000,1.072478,0.000000,1.075049,33.548112,4.488278,3.921569,87.722802,35.216972,0.387844,8.260,0.908079,3593.28,16.33,6.7753,28.491252,49.254923,11.762100,76.844237,25.590170,8.3567,1.3595,15.874794,151.535388
1,E06000002,Middlesbrough,33.973881,26.895990,66.532188,1.542756,29.727148,377.251648,9.474991,9.633600,0.798102,0.00000,0.711040,2.133121,4.269095,21.367826,159.546432,10.683913,8.547130,9.26,1.423032,1.423032,0.711040,0.711040,51.596741,6.167930,3.185509,102.972725,45.325860,0.703555,12.907,0.702882,3348.22,16.90,7.6342,32.644264,47.536469,11.322519,79.003433,21.776529,11.0908,2.7206,38.127593,167.054700
2,E06000003,Redcar and Cleveland,29.148571,13.845571,60.483284,0.623298,34.249569,273.996552,1.707846,8.710273,0.791662,0.00000,0.000000,0.000000,1.462865,17.724735,174.293226,8.862367,11.077959,5.17,0.731433,0.731433,0.000000,5.146870,44.191446,4.099241,2.113821,70.536439,38.450657,0.564981,7.923,0.832709,2826.70,9.56,7.1835,24.229035,44.094975,11.403048,81.725488,23.795458,10.6676,1.6482,27.994488,159.286643
3,E06000004,Stockton-on-Tees,25.833380,40.522949,52.173298,1.269739,44.068707,285.686798,3.045438,12.769310,0.747329,0.50894,0.000000,0.508940,1.014132,12.775895,133.380349,13.286931,6.643466,3.07,0.507066,1.521198,0.000000,1.017879,33.906193,9.693536,3.082550,72.481110,37.164935,0.572975,7.068,0.750743,3413.04,8.54,7.0901,20.670185,54.663153,10.918133,81.681734,22.951620,9.7462,0.9781,19.466967,138.577515
4,E06000005,Darlington,52.140556,4.655407,76.348671,1.263115,54.933800,255.116287,5.004441,9.173668,0.776001,0.00000,0.000000,2.820954,0.938386,26.503606,139.143934,1.893115,7.572459,1.89,0.000000,0.000000,3.761272,1.880636,34.350786,4.953044,2.555911,87.146438,38.933935,0.712441,10.710,0.917787,2314.26,10.83,6.3858,24.149011,50.733688,11.263320,85.933719,21.233568,16.8815,2.4737,22.300694,160.884642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,E07000200,Babergh,21.566830,4.313366,18.331806,0.741745,21.566830,185.474747,1.458624,6.951183,1.010644,0.00000,1.101394,1.101394,0.000000,14.525464,147.489329,1.117343,6.704060,1.12,0.000000,1.094080,9.912549,1.101394,16.824149,2.467582,2.121212,49.622201,15.870624,1.646785,18.526,0.934592,1762.74,NaN,9.3804,21.502115,59.974311,10.692612,67.595879,19.173536,11.0886,1.3656,11.626001,107.050539
196,E07000202,Ipswich,42.653645,2.206223,65.451286,1.858875,33.093346,403.003418,5.812193,8.061464,0.822900,0.00000,0.000000,0.000000,1.454207,8.093710,86.087648,5.886335,14.715837,2.94,0.000000,0.000000,7.221259,0.000000,34.212929,7.574153,2.928349,68.322287,29.673150,0.515439,29.905,0.676653,2190.72,4.84,10.0741,25.797279,51.893379,10.881856,73.088815,14.445032,13.6050,1.0847,14.830429,121.178832
197,E07000203,Mid Suffolk,24.795673,1.907360,21.934635,0.816475,24.795673,204.087479,0.324748,3.974092,0.957624,0.00000,0.000000,0.984804,1.951353,13.998040,145.979563,8.998740,20.997060,1.00,0.000000,0.975676,13.787263,0.000000,16.449042,2.209176,2.387268,47.540666,19.790439,0.787215,21.532,0.833020,1685.11,NaN,9.3510,17.070144,60.253823,10.774657,67.765822,16.659401,13.6055,1.0883,7.471232,96.117424
198,E07000207,Elmbridge,49.557262,5.101483,46.642132,1.462008,29.880116,115.876541,3.416301,7.293921,0.580432,0.00000,0.733251,0.000000,1.463850,15.064325,91.139164,24.856136,6.778946,0.00,0.000000,0.000000,5.866006,0.733251,14.849865,3.998684,1.982379,55.192269,15.270033,0.825707,14.906,0.604127

In [4]:
#merge with 2 other features (downloaded manually, since the API call failed to access them)
education = pd.read_csv('education.csv')
premature = pd.read_csv('premature_births.csv')
dataset = data.copy()
dataset = pd.merge(dataset, education[['Area Code','Value']], on='Area Code').rename(columns={'Value':'92199'})
dataset = pd.merge(dataset, premature[['Area Code','Value']], on='Area Code').rename(columns={'Value':'91743'})
dataset.to_csv('rawData.csv',index=False)

In [3]:
dataset.head()

,Area Code,Area Name,90729,90742,90759,90790,91046,91521,91361,92326,...,93077,93093,93347,93377,93439,93440,93563,40501,92199,91743
0,E06000001,Hartlepool,35.167740,11.722580,54.350143,0.875074,43.693253,346.348938,3.572398,9.310346,...,49.254923,11.762100,76.844237,25.590170,8.3567,1.3595,15.874794,151.535388,48.5130,95.0999
1,E06000002,Middlesbrough,33.973881,26.895990,66.532188,1.542756,29.727148,377.251648,9.474991,9.633600,...,47.536469,11.322519,79.003433,21.776529,11.0908,2.7206,38.127593,167.054700,53.4943,84.2472
2,E06000003,Redcar and Cleveland,29.148571,13.845571,60.483284,0.623298,34.249569,273.996552,1.707846,8.710273,...,44.094975,11.403048,81.725488,23.795458,10.6676,1.6482,27.994488,159.286643,54.1896,90.1389
3,E06000004,Stockton-on-Tees,25.833380,40.522949,52.173298,1.269739,44.068707,285.686798,3.045438,12.769310,...,54.663153,10.918133,81.681734,22.951620,9.7462,0.9781,19.466967,138.577515,58.3873,92.8039
4,E06000005,Darlington,52.140556,4.655407,76.348671,1.263115,54.933800,255.116287,5.004441,9.173668,...,50.733688,11.263320,85.933719,21.233568,16.8815,2.4737,22.300694,160.884642,55.8850,99.4443


In [4]:
#check for missing values
dataset.isnull().sum()

Area Code     0
Area Name     0
90729         0
90742         0
90759         0
90790         0
91046         0
91521         0
91361         0
92326         4
92670         0
92841         0
93119         0
93124         0
93316         0
93355         0
93356         0
93379         0
93380         0
93357         0
93456         0
93466         0
93515         0
93604         0
92621         0
11401         5
20101         0
40401         0
40701         3
90443         0
90669         0
91269         0
91409         5
92432        76
92924         0
93015         0
93077         0
93093         0
93347         0
93377         0
93439         0
93440         0
93563        41
40501         0
92199         0
91743         0
dtype: int64

In [5]:
#view dimensions
dataset.shape

(299, 46)

In [6]:
#drop column '92432': It's missing 25% of its data, so imputing values could cause more harm than good
#drop column '93563': I tried imputing these values, but even the parent area data was missing
dataset.drop(['92432','93563'], axis=1, inplace=True)

In [7]:
#To impute missing values, access geospatial data from FindThatPostCode API to determine parent area
#Add parent area to the dataframe and replace missing values with average parent-area values
area = []
parent_area = []
for i in dataset['Area Code']:
    print(i, 'attempting')
    url = 'https://findthatpostcode.uk/areas/{}.json'.format(i)
    response = requests.get(url)
    jsonOP = response.json()
    parse2 = jsonOP['data']['attributes']['parent']
    area.append(i)
    parent_area.append(parse2)
    print(i, ' completed')


E06000001 attempting
E06000001  completed
E06000002 attempting
E06000002  completed
E06000003 attempting
E06000003  completed
E06000004 attempting
E06000004  completed
E06000005 attempting
E06000005  completed
E06000006 attempting
E06000006  completed
E06000007 attempting
E06000007  completed
E06000008 attempting
E06000008  completed
E06000009 attempting
E06000009  completed
E06000010 attempting
E06000010  completed
E06000011 attempting
E06000011  completed
E06000012 attempting
E06000012  completed
E06000013 attempting
E06000013  completed
E06000014 attempting
E06000014  completed
E06000015 attempting
E06000015  completed
E06000016 attempting
E06000016  completed
E06000017 attempting
E06000017  completed
E06000018 attempting
E06000018  completed
E06000019 attempting
E06000019  completed
E06000020 attempting
E06000020  completed
E06000021 attempting
E06000021  completed
E06000022 attempting
E06000022  completed
E06000023 attempting
E06000023  completed
E06000024 attempting
E06000024  co

E07000200  completed
E07000202 attempting
E07000202  completed
E07000203 attempting
E07000203  completed
E07000207 attempting
E07000207  completed
E07000208 attempting
E07000208  completed
E07000209 attempting
E07000209  completed
E07000210 attempting
E07000210  completed
E07000211 attempting
E07000211  completed
E07000212 attempting
E07000212  completed
E07000213 attempting
E07000213  completed
E07000214 attempting
E07000214  completed
E07000215 attempting
E07000215  completed
E07000216 attempting
E07000216  completed
E07000217 attempting
E07000217  completed
E07000218 attempting
E07000218  completed
E07000219 attempting
E07000219  completed
E07000220 attempting
E07000220  completed
E07000221 attempting
E07000221  completed
E07000222 attempting
E07000222  completed
E07000223 attempting
E07000223  completed
E07000224 attempting
E07000224  completed
E07000225 attempting
E07000225  completed
E07000226 attempting
E07000226  completed
E07000227 attempting
E07000227  completed
E07000228 att

In [8]:
#store the retrieved data in dataframe
regions = pd.DataFrame(area, columns=['area'])
regions['parent_area'] = parent_area

In [9]:
#collect the names of the parent areas
parent_area2 = []
parent_name = []
for j in regions['parent_area'].unique():
    print(j, 'attempting')
    url = 'https://findthatpostcode.uk/areas/{}.json'.format(j)
    response = requests.get(url)
    jsonOP = response.json()
    parse2 = jsonOP['data']['attributes']['name']
    parent_area2.append(j)
    parent_name.append(parse2)
    print(j, ' completed')


E12000001 attempting
E12000001  completed
E12000002 attempting
E12000002  completed
E12000003 attempting
E12000003  completed
E12000004 attempting
E12000004  completed
E12000005 attempting
E12000005  completed
E12000009 attempting
E12000009  completed
E12000006 attempting
E12000006  completed
E12000008 attempting
E12000008  completed
E10000003 attempting
E10000003  completed
E10000006 attempting
E10000006  completed
E10000007 attempting
E10000007  completed
E10000008 attempting
E10000008  completed
E10000011 attempting
E10000011  completed
E10000012 attempting
E10000012  completed
E10000013 attempting
E10000013  completed
E10000014 attempting
E10000014  completed
E10000015 attempting
E10000015  completed
E10000016 attempting
E10000016  completed
E10000017 attempting
E10000017  completed
E10000018 attempting
E10000018  completed
E10000019 attempting
E10000019  completed
E10000020 attempting
E10000020  completed
E10000023 attempting
E10000023  completed
E10000024 attempting
E10000024  co

In [10]:
#store in dataframe and merge with subarea dataframe
parent_regions = pd.DataFrame(parent_area2, columns=['parent_area'])
parent_regions['parent_name'] = parent_name
regions=pd.merge(regions, parent_regions, on='parent_area')

In [11]:
regions

,area,parent_area,parent_name
0,E06000001,E12000001,North East
1,E06000002,E12000001,North East
2,E06000003,E12000001,North East
3,E06000004,E12000001,North East
4,E06000005,E12000001,North East
...,...,...,...
294,E09000029,E12000007,London
295,E09000030,E12000007,London
296,E09000031,E12000007,London
297,E09000032,E12000007,London


In [12]:
#join this to the main dataset
dataset = pd.merge(dataset, regions.rename(columns={'area':'Area Code'}), on='Area Code')

In [13]:
#Impute the missing values
cols_with_missing_vals = [x for x in dataset.columns.where(dataset.isnull().sum()!=0).unique()][1:]
for col in cols_with_missing_vals:
    dataset[col] = dataset.apply(lambda x: dataset[dataset['parent_area']==x['parent_area']][col].mean() if pd.isnull(x[col]) else x[col],axis=1)

In [14]:
#check
dataset.isnull().sum()

Area Code      0
Area Name      0
90729          0
90742          0
90759          0
90790          0
91046          0
91521          0
91361          0
92326          0
92670          0
92841          0
93119          0
93124          0
93316          0
93355          0
93356          0
93379          0
93380          0
93357          0
93456          0
93466          0
93515          0
93604          0
92621          0
11401          0
20101          0
40401          0
40701          0
90443          0
90669          0
91269          0
91409          5
92924          0
93015          0
93077          0
93093          0
93347          0
93377          0
93439          0
93440          0
40501          0
92199          0
91743          0
parent_area    0
parent_name    0
dtype: int64

In [15]:
#investigating the missing values
dataset[dataset['91409'].isnull()]

,Area Code,Area Name,90729,90742,90759,90790,91046,91521,91361,92326,...,93093,93347,93377,93439,93440,40501,92199,91743,parent_area,parent_name
80,E07000061,Eastbourne,48.391468,8.710464,76.458519,2.319423,41.616661,246.796478,3.546602,9.232640,...,11.285401,69.132564,20.335484,17.7124,2.2749,115.359942,57.9121,71.7355,E10000011,East Sussex
81,E07000062,Hastings,45.378914,8.643602,70.229271,2.425783,36.735310,332.778717,3.233583,9.698487,...,11.425779,73.750686,22.028225,12.3595,1.9854,146.154971,49.9478,77.7744,E10000011,East Sussex
82,E07000063,Lewes,37.672058,6.761652,78.241974,2.078924,13.523304,210.577148,3.568312,7.171606,...,10.884089,68.723384,20.987413,21.1930,0.2125,112.311429,60.0414,77.9903,E10000011,East Sussex
83,E07000064,Rother,17.577236,3.101865,20.679102,1.683766,22.747011,172.670502,1.046270,8.025928,...,10.811444,69.317081,24.063560,13.0729,1.5354,117.964634,59.4561,80.6230,E10000011,East Sussex
84,E07000065,Wealden,39.942730,2.458014,23.351133,0.874423,23.351133,121.671692,2.080771,6.859801,...,10.633818,68.083375,20.238356,13.6714,1.2642,110.977782,65.0458,73.9076,E10000011,East Sussex


In [16]:
#looking at the 3 neighboring parent regions shows similar average rates, whereas areas further away show 
#different average rates. So I'll average the averages of the 3 neighbors and use those for imputation
checker = dataset[dataset['parent_name'].isin(['East Sussex','West Sussex','Surrey','Kent'])]
print(checker.groupby('parent_name')['91409'].mean())
print(dataset.groupby('parent_name')['91409'].mean())

parent_name
East Sussex            NaN
Kent           1945.210833
Surrey         1949.654545
West Sussex    1922.181429
Name: 91409, dtype: float64
parent_name
Cambridgeshire              2376.698000
Cumbria                     2125.388333
Derbyshire                  2423.188750
Devon                       1665.525000
East Midlands               2543.302500
East Sussex                         NaN
East of England             2426.346667
Essex                       2040.374167
Gloucestershire             1905.471667
Hampshire                   2048.151818
Hertfordshire               2282.008000
Kent                        1945.210833
Lancashire                  2603.912500
Leicestershire              1983.334286
Lincolnshire                1969.824286
London                      2534.609375
Norfolk                     2299.384286
North East                  3036.242500
North West                  2904.048571
North Yorkshire             2168.220000
Nottinghamshire             2374.110000


In [17]:
#populate the missing values
replacement = round(checker.groupby('parent_name')['91409'].mean().mean(),2)
dataset['91409'] = dataset['91409'].apply(lambda x: replacement if pd.isnull(x) else x)

In [18]:
#convert indicator codes to descriptions for interpretability
dataset = dataset.rename(columns={'40501':'cancer_death_rate','90729':'gen_warts','90742':'syphilis','90759':'gonorrhea','90790':'HIV',
                           '91046':'herpes','91521':'chlamydia','91361':'TB','92326':'comm_disease_flu',
                           '92670':'antibiotics','92841':'typhoid','93119':'measles','93124':'hepatitis',
                           '93316':'STEC','93355':'salmonella','93356':'campylobacter','93379':'giardia',
                           '93380':'cryptosporidium','93357':'shigella','93456':'listeria','93466':'mumps',
                           '93515':'pertusis','93604':'IMD','92621':'mental','11401':'noise','20101':'low_birth_wt',
                          '40401':'cardiovascular','40701':'respiratory','90443':'osteoporosis','90669':'indoor_air',
                          '91269':'RA','91409':'alcohol','91743':'premature','92199': 'education',
                          '92924':'pollution','93015':'inactive','93077':'Five_a_day','93093':'back_pain',
                          '93347':'diabetes','93377':'MSK','93439':'walking','93440':'biking'})

In [19]:
dataset.to_csv('cleanData.csv',index=False)